In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
df = pd.read_csv("/kaggle/input/delhi-house-price-prediction/MagicBricks.csv")
df.head()

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.Bathroom.unique()

In [ ]:
df.Furnishing.unique()

In [ ]:
df.Parking.unique()

In [ ]:
df.Type.unique()

In [ ]:
df.Per_Sqft.unique()

In [ ]:
df1 = df.drop(['Furnishing','Transaction','Type','Status'], axis = 'columns')

In [ ]:
df1

In [ ]:
len(df1)

In [ ]:
df1.isnull().sum()

In [ ]:
df2 = df1.dropna()

In [ ]:
df2.info()

In [ ]:
df2.head()

In [ ]:
df3 = df2[~(df2.Bathroom >4)]

In [ ]:
df3.info()

In [ ]:
round(df3.Area)

In [ ]:
df3.head()

In [ ]:
df3[df3.BHK < df3.Bathroom]

In [ ]:
df4 = df3[~(df3.Parking >4)]

In [ ]:
df4.info()

In [ ]:
location_stat = df4.groupby(['Locality'])['Locality'].agg('count').sort_values(ascending = False)

In [ ]:
location_stat_less_than4 = location_stat[location_stat ==2]

In [ ]:
len(location_stat_less_than4)

In [ ]:
df4.Locality = df4['Locality'].apply(lambda x : 'other' if x in location_stat_less_than4 else x)

In [ ]:
df4.groupby(['Locality'])['Locality'].agg('count').sort_values(ascending = False)

In [ ]:
df4.info()

In [ ]:
df5 = df4[~(df4.Area/df4.BHK <300)]

In [ ]:
df5

In [ ]:
df4[(df4.Area/df4.BHK <300)]

In [ ]:
df5.Per_Sqft.describe()

In [ ]:
df6 = df5[~(df5.Per_Sqft < 2500)]

In [ ]:
df6.Per_Sqft.describe()

In [ ]:
plt.hist(df6.Bathroom, width = 0.4)
plt.xlabel('Number of Bathroom')
plt.ylabel('Count')

In [ ]:
plt.hist(df6.Parking, width = 0.4)
plt.xlabel('Number of Bathroom')
plt.ylabel('Count')

In [ ]:
df6[df6.BHK < (df6.Parking)]

In [ ]:
df6.head()

In [ ]:
df7 = df6.drop(['Per_Sqft'],axis = 'columns')

In [ ]:
df7

In [ ]:
df7['BHK'] = df7['BHK'].astype(float)
df7['Price'] = df7['Price'].astype(float)

In [ ]:
df7.dtypes

In [ ]:
df7.to_csv('df7.csv')

In [ ]:
dummies = pd.get_dummies(df7.Locality,dtype=float)

In [ ]:
dummies

In [ ]:
df8 = pd.concat([df7,dummies.drop('other',axis='columns')],axis = 'columns')
df8

In [ ]:
df9 = df8.drop('Locality',axis = 'columns')
df9

In [ ]:
x= df9.drop('Price',axis = 'columns')

In [ ]:
y = df9.Price

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train, y_test = train_test_split(x,y,test_size= 0.2, random_state =10)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
x_train = x_train.fillna(x_train.mean())
y_train= y_train.fillna(y_train.mean())
x_test= x_test.fillna(x_train.mean())
y_test = y_test.fillna(y_train.mean())
x = x.fillna(x.mean())
y = y.fillna(y.mean())

In [ ]:
model.fit(x_train,y_train)

In [ ]:
model.score(x_test,y_test)

In [ ]:
def predict_price(location,sqft,bath,bhk,park):
    loc_index = np.where(x.columns==location)[0][0]
    
    X= np.zeros(len(x.columns))
    X[0] = sqft
    X[1] = bath
    X[2] = bhk
    X[3] = park
    if loc_index >= 0:
        X[loc_index] =1
        
    return model.predict([X])[0]


In [ ]:
predict_price('Rohini Sector 24',1300,3,4,1)

In [ ]:
predict_price('Chittaranjan Park',1700,5,4,2)